In [27]:
import pandas as pd
from tqdm import tqdm
from glob import glob
import numpy as np

In [28]:
cl = [e[3:] for e in "223SLHC,224HST,225HCT,226MCV,227MCH,228MCHC,234SLTC,246SLBC,233RDW-CV,indate".split(",")]
cl

['SLHC', 'HST', 'HCT', 'MCV', 'MCH', 'MCHC', 'SLTC', 'SLBC', 'RDW-CV', 'ate']

In [29]:
# for name in ['TBM']:
#     files = glob(f"../datasets/{name}/*")
#     df_list = [pd.read_csv(e).set_index("PID")[cl] for e in files]
#     df = pd.concat(df_list)
#     df.to_csv(f"../datasets/TONG_HOP/{name}.csv")

In [30]:
files = glob(f"../datasets/TONG_HOP/*")
df_list = [pd.read_csv(e).set_index("PID") for e in files]

In [31]:
files

['../datasets/TONG_HOP\\DIEN_DI.csv',
 '../datasets/TONG_HOP\\DI_TRUYEN.csv',
 '../datasets/TONG_HOP\\pid.csv',
 '../datasets/TONG_HOP\\SINH_HOA.csv',
 '../datasets/TONG_HOP\\TBM.csv']

In [32]:
pid_df = df_list[2]
pid_df.index = pid_df.index.astype(int)

tbm_df = df_list[-1].assign(inyear=df_list[-1].indate.str.split('-').str[0])
sinh_hoa_df = df_list[-2].assign(inyear=df_list[-2].indate.str.split('-').str[0])
sinh_hoa_df = sinh_hoa_df[['FE', 'FERRITIN', 'inyear']]
dien_di_df  = df_list[0].assign(inyear=df_list[0].indate.str.split('-').str[0])
dien_di_df = dien_di_df[['HBA1', 'HBE', 'HBF', 'HBA2', 'HBH', 'HBB', 'inyear']]

In [33]:
tong_hop_df = pid_df.merge(tbm_df, how='left', left_index=True, right_index=True)
tong_hop_df = tong_hop_df.merge(sinh_hoa_df, how='outer', left_on=['PID', 'inyear'], right_on=['PID', 'inyear'])
tong_hop_df = tong_hop_df.merge(dien_di_df, how='outer', left_on=['PID', 'inyear'], right_on=['PID', 'inyear'])
tong_hop_df

,SLHC,HST,HCT,MCV,MCH,MCHC,SLTC,SLBC,RDW-CV,indate,inyear,FE,FERRITIN,HBA1,HBE,HBF,HBA2,HBH,HBB
PID,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123456,5.32,118.0,0.383,71.9,22.2,309.0,264.0,7.12,NaN,2016-08-29,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123456,2.00,2.0,2.000,2.0,2.0,2.0,2.0,2.00,2,2019-06-14,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000544,2.93,64.0,0.228,77.9,22.0,282.0,167.0,9.81,NaN,2016-04-23,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008410,3.87,142.0,0.421,108.8,36.8,338.0,113.0,3.98,14.0,2018-10-20,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920045023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,NaN,70.8,29.2,NaN,NaN,NaN,NaN
1920054399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,NaN,98.3,NaN,NaN,1.7,NaN,NaN
1920054400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,NaN,98.5,NaN,NaN,1.5,NaN,NaN


In [34]:
di_truyen_df = df_list[1]
di_truyen_df.index = di_truyen_df.index.astype(int)
di_truyen_df = di_truyen_df[~di_truyen_df.index.duplicated(keep='first')]

In [46]:
from mapping import mp, alpha_names, beta_names, pattern

b_head = []
a_head = []
def process_header(df):
    target_lb = list(pattern.keys())

    for idx, e in enumerate(df.columns.values):
        for k in alpha_names:
            if k.lower() in str(e).lower():
                df.columns.values[idx] = f'{k}_{idx}'
                a_head.append(df.columns.values[idx])
                break
        for k in beta_names:
            if k.lower() in str(e).lower():
                df.columns.values[idx] = f'{k}_{idx}'
                b_head.append(df.columns.values[idx])
                break 
                
    df.columns = df.columns.values
    print(df.columns)

In [47]:
process_header(di_truyen_df)

Index(['4.2_0', 'SEA_1', 'CD95_2', 'CD41/CD42', 'CD17_4', 'Alpha.Thalassemia',
       'indate', 'SEA_7', 'codon_8', 'codon_9', 'codon_10', '71/72_11',
       'IVS_12', '-28_13', 'IVS_14', 'IVS_15', '41/42_16', 'HbQs_17', 'FIL_18',
       '2delT_19', '3.7_20', 'HbCS_21', '4.2_22', 'THAI_23', '-88_24',
       'thal -90_25', '71/72_26', '- 28', 'IVS_28', 'IVS_29', 'IVS_30', '- 29',
       'codon_32', 'codon_33', 'codon_34', 'codon_35', '- 31', 'codon_37',
       '89/90_38', 'codon_39', 'IVS_40', 'codon_41', 'codon_42', '41/42_43',
       'codon_44', 'codon_45', 'SEA_46', 'cd _47', 'cd _48', '4.2_49',
       'cd _50', 'cd _51', '3.7_52', '-20.5kb', '--MED', 'cd _55', 'cd _56',
       '3.7_57', 'cd 19_58', 'IVS_59', 'cd _60', 'cd _61', 'a2 poly A-1',
       'a2 poly A-2', 'codon_64', '-88_65', 'bệnh PV/ET/PMF)', 'codon_67',
       'thal -90_68', 'FIL_69', 'cap+1'],
      dtype='object')


In [37]:
keep_columns = [e for e in di_truyen_df.columns.values if '_' in e]

In [38]:
keep_idx = di_truyen_df[keep_columns].dropna(how='all').index

In [39]:
result_df = di_truyen_df[['indate'] + keep_columns].loc[keep_idx]

In [52]:
set(result_df.columns) - (set(a_head) | set(b_head)), a_head


({'indate'},
 ['4.2_0',
  'SEA_1',
  'SEA_7',
  'HbQs_17',
  'FIL_18',
  '2delT_19',
  '3.7_20',
  'HbCS_21',
  '4.2_22',
  'THAI_23',
  'SEA_46',
  '4.2_49',
  '3.7_52',
  '3.7_57',
  'FIL_69'])

In [53]:
result_df = result_df.replace(regex='((Dương)|(Duong)|(DƯƠNG)|(duong)|(DUONG))+', value=-2)

In [54]:
result_df.columns

Index(['indate', '4.2_0', 'SEA_1', 'CD95_2', 'CD17_4', 'SEA_7', 'codon_8',
       'codon_9', 'codon_10', '71/72_11', 'IVS_12', '-28_13', 'IVS_14',
       'IVS_15', '41/42_16', 'HbQs_17', 'FIL_18', '2delT_19', '3.7_20',
       'HbCS_21', '4.2_22', 'THAI_23', '-88_24', 'thal -90_25', '71/72_26',
       'IVS_28', 'IVS_29', 'IVS_30', 'codon_32', 'codon_33', 'codon_34',
       'codon_35', 'codon_37', '89/90_38', 'codon_39', 'IVS_40', 'codon_41',
       'codon_42', '41/42_43', 'codon_44', 'codon_45', 'SEA_46', 'cd _47',
       'cd _48', '4.2_49', 'cd _50', 'cd _51', '3.7_52', 'cd _55', 'cd _56',
       '3.7_57', 'cd 19_58', 'IVS_59', 'cd _60', 'cd _61', 'codon_64',
       '-88_65', 'codon_67', 'thal -90_68', 'FIL_69'],
      dtype='object')

In [55]:
a_df = result_df[a_head]
b_df = result_df[b_head]

In [56]:
lb = np.any(result_df == -2, axis=1).astype(int)
alb = np.any(a_df == -2, axis=1).astype(int)
blb = np.any(b_df == -2, axis=1).astype(int)

In [57]:
result_df = result_df.assign(label = lb, alpha=alb, beta=blb)

In [58]:
pt_df = result_df[['indate','alpha', 'beta', 'label']]

In [59]:
pt_df[(((pt_df.alpha + pt_df.beta) > 0).astype(int) != pt_df.label.astype(int))]

,indate,alpha,beta,label
PID,,,,


In [60]:
pt_df.loc[7008235]

indate    ﻿2013-03-26
alpha               0
beta                0
label               0
Name: 7008235, dtype: object

In [64]:
tong_hop_df = tong_hop_df.merge(pt_df, how='left', left_on=['PID'], right_on=['PID'])


In [67]:
mask_df = tong_hop_df[['SLHC', 'HST', 'HCT', 'MCV', 'MCH', 'MCHC', 'SLTC', 'SLBC', 'RDW-CV']]
tong_hop_df[np.any(mask_df, axis=1)].to_csv('../datasets/tong_hop_p.csv')

In [1]:
tong_hop_df

NameError: name 'tong_hop_df' is not defined

In [63]:
pt_df.la

,indate,alpha,beta,label
PID,,,,
7008235,﻿2013-03-26,0,0,0
8004477,2013-03-26,0,0,0
9002987,2013-03-08,0,1,1
9004809,2013-03-11,0,0,0
9006069,2013-03-12,0,0,0
...,...,...,...,...
2020109668,2020-01-31,1,0,1
2020109669,2020-01-31,1,0,1
2020109670,2020-01-31,1,0,1


In [ ]:
pt_df